In [ ]:
!pip install ipyleaflet
import ipyleaflet
import ipywidgets
from ipywidgets import IntSlider

In [154]:
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.animation as animation
import cartopy.crs as ccrs
from cartopy.feature import NaturalEarthFeature
import xarray as xr
import geopandas as gpd
import numpy as np
import folium
from shapely.geometry import mapping

In [155]:
co2_emissions_csv = pd.read_csv('data/co2_with_geometries.csv') #data from "Our world in data"
co2_emissions_csv

,Entity,Code,Year,Annual CO₂ emissions (per capita)
0,Afghanistan,AFG,1949,0.001922
1,Afghanistan,AFG,1950,0.011266
2,Afghanistan,AFG,1951,0.012098
3,Afghanistan,AFG,1952,0.011946
4,Afghanistan,AFG,1953,0.013685
...,...,...,...,...
26910,Zimbabwe,ZWE,2017,0.650533
26911,Zimbabwe,ZWE,2018,0.783639
26912,Zimbabwe,ZWE,2019,0.723861
26913,Zimbabwe,ZWE,2020,0.676970


In [156]:
#filter by USA
co2_USA = co2_emissions_csv[co2_emissions_csv["Code"] == 'USA']
co2_USA

,Entity,Code,Year,Annual CO₂ emissions (per capita)
25383,United States,USA,1800,0.042136
25384,United States,USA,1801,0.043749
25385,United States,USA,1802,0.046464
25386,United States,USA,1803,0.046753
25387,United States,USA,1804,0.051548
...,...,...,...,...
25600,United States,USA,2017,15.800776
25601,United States,USA,2018,16.187923
25602,United States,USA,2019,15.730885
25603,United States,USA,2020,14.037217


In [157]:
#rename "codes" to "geometry"
co2_USA = co2_USA.rename(columns={'Code': 'geometry'}, inplace=False)
co2_USA

,Entity,geometry,Year,Annual CO₂ emissions (per capita)
25383,United States,USA,1800,0.042136
25384,United States,USA,1801,0.043749
25385,United States,USA,1802,0.046464
25386,United States,USA,1803,0.046753
25387,United States,USA,1804,0.051548
...,...,...,...,...
25600,United States,USA,2017,15.800776
25601,United States,USA,2018,16.187923
25602,United States,USA,2019,15.730885
25603,United States,USA,2020,14.037217


In [158]:
import shapely

In [159]:
from shapely.geometry import Polygon

In [160]:
#coordinates from google
usa_coords = [
    (-124.848974, 49.000000),  # NW corner
    (-124.848974, 24.396308),  # SW corner
    (-66.885444, 24.396308),   # SE corner
    (-66.885444, 49.384358),   # NE corner
    (-124.848974, 49.384358)]  # NW corner (repeat to close polygon)

In [161]:
# create a Shapely polygon object from the coordinates
polygon = Polygon(usa_coords)

In [162]:
#replace the geometry column for USA variable with the geometry
co2_USA['geometry'] = co2_USA['geometry'].replace('USA', polygon)

In [163]:
geojson = mapping(polygon)

In [164]:
#rename "codes" to "geometry"
co2_USA = co2_USA.rename(columns={'Annual CO₂ emissions (per capita)': 'emissions_per_capita'}, inplace=False)
co2_USA

,Entity,geometry,Year,emissions_per_capita
25383,United States,"POLYGON ((-124.848974 49, -124.848974 24.39630...",1800,0.042136
25384,United States,"POLYGON ((-124.848974 49, -124.848974 24.39630...",1801,0.043749
25385,United States,"POLYGON ((-124.848974 49, -124.848974 24.39630...",1802,0.046464
25386,United States,"POLYGON ((-124.848974 49, -124.848974 24.39630...",1803,0.046753
25387,United States,"POLYGON ((-124.848974 49, -124.848974 24.39630...",1804,0.051548
...,...,...,...,...
25600,United States,"POLYGON ((-124.848974 49, -124.848974 24.39630...",2017,15.800776
25601,United States,"POLYGON ((-124.848974 49, -124.848974 24.39630...",2018,16.187923
25602,United States,"POLYGON ((-124.848974 49, -124.848974 24.39630...",2019,15.730885
25603,United States,"POLYGON ((-124.848974 49, -124.848974 24.39630...",2020,14.037217


In [198]:
# Create a Folium map centered on the USA
m = folium.Map(location=[37, -95], zoom_start=4)
m

In [191]:
# Define a function to add a GeoJSON layer to the map for the specified year
def add_geojson_layer(year):
    # Filter the data for the specified year
    data = co2_USA[co2_USA['Year'] == year]

In [199]:
# Create a GeoJSON layer for the filtered data
geojson_layer = folium.GeoJson(
        co2_USA['geometry'],
        name='CO2 Emissions',
        style_function=lambda feature: {
            'fillColor': cmap(feature['properties']['emissions_per_capita']),
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        }
    )
geojson_layer.add_to(m)

ValueError: Cannot render objects with any missing geometries: 25383    POLYGON ((-124.848974 49, -124.848974 24.39630...
25384    POLYGON ((-124.848974 49, -124.848974 24.39630...
25385    POLYGON ((-124.848974 49, -124.848974 24.39630...
25386    POLYGON ((-124.848974 49, -124.848974 24.39630...
25387    POLYGON ((-124.848974 49, -124.848974 24.39630...
                               ...                        
25600    POLYGON ((-124.848974 49, -124.848974 24.39630...
25601    POLYGON ((-124.848974 49, -124.848974 24.39630...
25602    POLYGON ((-124.848974 49, -124.848974 24.39630...
25603    POLYGON ((-124.848974 49, -124.848974 24.39630...
25604    POLYGON ((-124.848974 49, -124.848974 24.39630...
Name: geometry, Length: 222, dtype: object

In [149]:
slider = IntSlider(min=1800, max=2021, step=1, value=1921)
slider

IntSlider(value=1921, max=2021, min=1800)

In [183]:
# Create a Folium map centered on the USA
#m = folium.Map(location=[37, -95], zoom_start=4)

# import folium
# m = folium.Map(location=[34.0522, -118.2437], zoom_start=5, control_scale=True,tiles=' cartodbpositron',
#                prefer_canvas=True)
# folium.GeoJson(santa_m,
#                 name='USA Carbon Emission',
#                 style_function=lambda x: {'color':'black','fillColor':'red','weight':1},
#                 highlight_function=lambda x: {'weight':3, 'color':'black'},
#                 zoom_on_click=True
#                ).add_to(m)
# folium.LayerControl().add_to(m)
# m